Loading Data in Parallel

In [4]:
import scanpy as sc
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple, List, Dict, Iterator
from collections import defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.sparse import csc_matrix
import time
import gc  # For garbage collection
import os


class OptimizedGeneDataProcessor:
    """Processes gene expression data for multiple target genes efficiently."""

    def __init__(self):
        self.gene_data = {}
        self.sample_names = None
        self.related_genes = {}
        self.expr_matrix = None
        self.gene_names = None
        self.gene_name_to_idx = {}
        self.gene_network = defaultdict(list)
        self.valid_targets = []
        self.start_time = None

    def load_data(self, expression_file: Path, network_file: Path):
        """
        Loads expression and network data once for all target genes.

        Args:
            expression_file: Path to h5ad expression data file
            network_file: Path to network TSV file
        """
        self.start_time = time.time()
        print("Loading expression data...")
        adata = sc.read_h5ad(expression_file)
        self.expr_matrix = (
            csc_matrix(adata.X) if not hasattr(adata.X, "tocsc") else adata.X.tocsc()
        )
        self.gene_names = np.array(adata.var_names.tolist())
        self.sample_names = np.array(adata.obs_names.tolist())
        self.gene_name_to_idx = {gene: idx for idx, gene in enumerate(self.gene_names)}

        print("Loading network data...")
        network_df = pd.read_csv(network_file, sep="\t")
        source_col, target_col = network_df.columns[:2]

        for _, row in network_df.iterrows():
            if (
                row[target_col] in self.gene_name_to_idx
                and row[source_col] in self.gene_name_to_idx
            ):
                self.gene_network[row[target_col]].append(row[source_col])

        self.valid_targets = list(self.gene_network.keys())
        print(f"Found {len(self.valid_targets)} valid target genes in the network.")

    def get_all_target_genes(self) -> List[str]:
        """Returns list of all valid target genes in the network."""
        return self.valid_targets

    def prepare_training_data_for_gene(
        self, target_gene: str
    ) -> Tuple[np.ndarray, np.ndarray, Dict, List[str]]:
        """
        Prepares training data for a specific target gene.

        Args:
            target_gene: Name of target gene

        Returns:
            Tuple of (input matrix, target values, model config, related genes list)
        """
        related_genes = self.gene_network.get(target_gene, [])
        if not related_genes:
            print(f"Warning: No related genes found for {target_gene}")
            return None, None, None, []

        target_idx = self.gene_name_to_idx[target_gene]
        related_indices = [self.gene_name_to_idx[gene] for gene in related_genes]

        try:
            X = self.expr_matrix[:, related_indices].toarray()
            y = self.expr_matrix[:, target_idx].toarray().flatten()

            config = {"width": [X.shape[1], 2, 1], "grid": 5, "k": 4, "seed": 42}

            return X.astype(np.float32), y.astype(np.float32), config, related_genes
        except MemoryError:
            print(
                f"Memory error processing gene {target_gene} with {len(related_genes)} related genes"
            )
            return None, None, None, []

    def prepare_all_training_data(
        self,
    ) -> Iterator[Tuple[str, np.ndarray, np.ndarray, Dict, List[str]]]:
        """
        Generator that yields prepared training data for all target genes.
        """
        for target_gene in tqdm(self.valid_targets, desc="Processing genes"):
            X, y, config, related_genes = self.prepare_training_data_for_gene(
                target_gene
            )
            if X is not None:
                yield target_gene, X, y, config, related_genes

    def process_in_batches(
        self, batch_size: int = 100, output_dir: str = None
    ) -> Dict[str, Dict]:
        """
        Process genes in memory-efficient batches and optionally save to disk.

        Args:
            batch_size: Number of genes to process in each batch
            output_dir: If provided, save data to this directory instead of returning

        Returns:
            Dictionary of processed gene data or info about saved files
        """
        if not self.valid_targets:
            raise ValueError("No data loaded. Call load_data() first.")

        expected_count = len(self.valid_targets)
        processed_count = 0
        error_genes = []
        processed_genes = {}

        # Create output directory if needed
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            data_dir = os.path.join(output_dir, "gene_data")
            os.makedirs(data_dir, exist_ok=True)

        self.start_time = time.time()
        print(f"Processing {expected_count} genes in batches of {batch_size}")

        # Process in batches
        for batch_start in range(0, expected_count, batch_size):
            batch_end = min(batch_start + batch_size, expected_count)
            print(
                f"\nBatch {batch_start//batch_size + 1}: Processing genes {batch_start+1}-{batch_end} of {expected_count}"
            )

            # Get the batch of genes
            batch_genes = self.valid_targets[batch_start:batch_end]

            # Process genes in this batch
            batch_data = {}
            for gene in tqdm(batch_genes, desc="Processing batch"):
                try:
                    X, y, config, related_genes = self.prepare_training_data_for_gene(
                        gene
                    )
                    if X is not None:
                        processed_count += 1

                        if output_dir:
                            # Save to disk instead of keeping in memory
                            gene_path = os.path.join(data_dir, f"{gene}")
                            os.makedirs(gene_path, exist_ok=True)

                            # Save arrays and metadata
                            np.save(os.path.join(gene_path, "X.npy"), X)
                            np.save(os.path.join(gene_path, "y.npy"), y)

                            # Save config and related genes as JSON
                            with open(
                                os.path.join(gene_path, "metadata.json"), "w"
                            ) as f:
                                import json

                                json.dump(
                                    {
                                        "config": config,
                                        "related_genes": related_genes,
                                        "X_shape": X.shape,
                                        "y_shape": y.shape,
                                    },
                                    f,
                                )

                            # Just store path info in memory
                            processed_genes[gene] = {
                                "path": gene_path,
                                "X_shape": X.shape,
                                "y_shape": y.shape,
                                "num_related_genes": len(related_genes),
                            }
                        else:
                            # Store in memory
                            batch_data[gene] = {
                                "X": X,
                                "y": y,
                                "config": config,
                                "related_genes": related_genes,
                            }
                except Exception as e:
                    print(f"Error processing gene {gene}: {e}")
                    error_genes.append((gene, f"Processing error: {str(e)}"))

            # Update processed genes dictionary if keeping in memory
            if not output_dir:
                processed_genes.update(batch_data)

            # Print progress
            elapsed = time.time() - self.start_time
            print(f"Progress: {processed_count}/{expected_count} genes processed")
            print(f"Elapsed time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
            print(
                f"Estimated remaining time: {(elapsed/processed_count)*(expected_count-processed_count)/60:.1f} minutes"
            )

            # Force garbage collection between batches
            gc.collect()

        # Print final summary
        elapsed_time = time.time() - self.start_time
        summary = {
            "processed_count": processed_count,
            "expected_count": expected_count,
            "error_count": len(error_genes),
            "total_time_seconds": elapsed_time,
            "errors": error_genes,
        }

        print(f"\nProcessing complete")
        print(f"Total genes processed: {processed_count}/{expected_count}")
        print(f"Genes with errors: {len(error_genes)}")
        print(f"Total time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

        if error_genes:
            print("\nFirst 10 genes with errors:")
            for gene, error in error_genes[:10]:
                print(f"  {gene}: {error}")
            if len(error_genes) > 10:
                print(f"  ... and {len(error_genes) - 10} more")

        if output_dir:
            # Save summary to the output directory
            with open(os.path.join(output_dir, "processing_summary.json"), "w") as f:
                import json

                json.dump(summary, f, indent=2)
            print(f"Data saved to {output_dir}")

            # Create a manifest file for easy loading
            with open(os.path.join(output_dir, "gene_manifest.json"), "w") as f:
                json.dump(processed_genes, f)

        return {"genes": processed_genes, "summary": summary}

    def load_gene_data(self, gene, data_dir):
        """
        Load a single gene's data from disk.

        Args:
            gene: Name of the gene
            data_dir: Directory where data is stored

        Returns:
            Tuple of (X, y, config, related_genes)
        """
        gene_path = os.path.join(data_dir, "gene_data", gene)

        # Load arrays
        X = np.load(os.path.join(gene_path, "X.npy"))
        y = np.load(os.path.join(gene_path, "y.npy"))

        # Load metadata
        with open(os.path.join(gene_path, "metadata.json"), "r") as f:
            import json

            metadata = json.load(f)

        return X, y, metadata["config"], metadata["related_genes"]

Loading Them

In [5]:
from pathlib import Path


# Initialize and load data
processor = OptimizedGeneDataProcessor()
processor.load_data(Path("Data/expression_data1.h5ad"), Path("Data/net_grn.tsv"))

# # For smaller datasets (keep in memory)
# results = processor.process_in_batches(batch_size=100)
# gene_data = results["genes"]  # Dictionary with all processed gene data

# OR for larger datasets (save to disk)
results = processor.process_in_batches(batch_size=100, output_dir="processed_gene_data")

Loading expression data...
Loading network data...
Found 23427 valid target genes in the network.
Processing 23427 genes in batches of 100

Batch 1: Processing genes 1-100 of 23427


Processing batch: 100%|██████████| 100/100 [00:02<00:00, 49.71it/s]


Progress: 100/23427 genes processed
Elapsed time: 2.1 seconds (0.0 minutes)
Estimated remaining time: 8.0 minutes

Batch 2: Processing genes 101-200 of 23427


Processing batch: 100%|██████████| 100/100 [00:01<00:00, 50.50it/s]


Progress: 200/23427 genes processed
Elapsed time: 4.3 seconds (0.1 minutes)
Estimated remaining time: 8.3 minutes

Batch 3: Processing genes 201-300 of 23427


...

Progress: 23300/23427 genes processed
Elapsed time: 656.6 seconds (10.9 minutes)
Estimated remaining time: 0.1 minutes

Batch 234: Processing genes 23301-23400 of 23427


Processing batch: 100%|██████████| 100/100 [00:03<00:00, 32.13it/s]


Progress: 23400/23427 genes processed
Elapsed time: 659.9 seconds (11.0 minutes)
Estimated remaining time: 0.0 minutes

Batch 235: Processing genes 23401-23427 of 23427


Processing batch: 100%|██████████| 27/27 [00:01<00:00, 26.18it/s]


Progress: 23427/23427 genes processed
Elapsed time: 661.1 seconds (11.0 minutes)
Estimated remaining time: 0.0 minutes

Processing complete
Total genes processed: 23427/23427
Genes with errors: 0
Total time: 661.21 seconds (11.02 minutes)
Data saved to processed_gene_data


In [7]:
import json
# Load the gene manifest file that was created during processing
with open(os.path.join("processed_gene_data", "gene_manifest.json"), "r") as f:
    processed_genes = json.load(f)

# Loop through the genes
for gene in processed_genes:
    X, y, config, related_genes = processor.load_gene_data(gene, "processed_gene_data")
    print(
        f"Loaded data for {gene}: X shape {X.shape}, y shape {y.shape}, config {config}"
    )

Loaded data for BX293535.1: X shape (17041, 141), y shape (17041,), config {'width': [141, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for WNT4: X shape (17041, 79), y shape (17041,), config {'width': [79, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for ARL14: X shape (17041, 151), y shape (17041,), config {'width': [151, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for IGLV1-47: X shape (17041, 105), y shape (17041,), config {'width': [105, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for GPRC5D: X shape (17041, 77), y shape (17041,), config {'width': [77, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for SLC6A20: X shape (17041, 108), y shape (17041,), config {'width': [108, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for HIST1H3C: X shape (17041, 118), y shape (17041,), config {'width': [118, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}
Loaded data for FGFBP2: X shape (17041, 90), y shape (17041,), config {'width': [90, 2, 1], 'grid': 5, 'k': 4, 'seed': 42}


In [29]:
import os
import json
import time
import numpy as np
import torch
import traceback
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


def print_gpu_memory():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        print(
            f"GPU Memory: Total={total_memory:.2f}GB, Allocated={allocated:.2f}GB, Reserved={reserved:.2f}GB"
        )

Using device: cuda


In [31]:
class GeneDataLoader:
    def __init__(self, data_dir):
        self.data_dir = data_dir

    def load_gene_data(self, gene):
        gene_path = os.path.join(self.data_dir, "gene_data", gene)

        X = np.load(os.path.join(gene_path, "X.npy"))
        y = np.load(os.path.join(gene_path, "y.npy"))

        with open(os.path.join(gene_path, "metadata.json"), "r") as f:
            metadata = json.load(f)

        return X, y, metadata["config"], metadata["related_genes"]

In [32]:
class GeneDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [33]:
from kan import *

In [34]:
def r2_score(y_true, y_pred):
    total_sum_squares = torch.sum((y_true - torch.mean(y_true)) ** 2)
    residual_sum_squares = torch.sum((y_true - y_pred) ** 2)
    return 1 - (residual_sum_squares / (total_sum_squares + 1e-8))


def rmse(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2))


def mae(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))

In [39]:
def train_kan_models_parallel(
    gene_list,
    data_dir,
    output_dir,
    batch_size=32,
    max_models=4,
    epochs=50,
    patience=10,
    lr=0.001,
):
    os.makedirs(output_dir, exist_ok=True)
    results = {}
    gene_loader = GeneDataLoader(data_dir)

    batch_pbar = tqdm(total=len(gene_list), desc="Overall Progress")

    for batch_start in range(0, len(gene_list), max_models):
        batch_end = min(batch_start + max_models, len(gene_list))
        batch_genes = gene_list[batch_start:batch_end]

        (
            models,
            optimizers,
            criterions,
            loaders,
            gene_dirs,
            best_val_losses,
            patience_counters,
        ) = ([], [], [], [], [], [], [])
        histories = []

        for gene in batch_genes:
            try:
                print(f"\nPreparing model for gene: {gene}")

                gene_dir = os.path.join(output_dir, gene)
                os.makedirs(gene_dir, exist_ok=True)

                X, y, config, _ = gene_loader.load_gene_data(gene)
                X_train, X_val, y_train, y_val = train_test_split(
                    X, y, test_size=0.2, random_state=42
                )

                train_loader = DataLoader(
                    GeneDataset(X_train, y_train), batch_size=batch_size, shuffle=True
                )
                val_loader = DataLoader(
                    GeneDataset(X_val, y_val), batch_size=batch_size
                )

                model = KAN(width=config["width"],grid=config["grid"],k=config["k"],seed=config["seed"],device=device).to(
                    device
                )
                optimizer = torch.optim.Adam(model.parameters(), lr=lr)
                criterion = nn.MSELoss()

                models.append(model)
                optimizers.append(optimizer)
                criterions.append(criterion)
                loaders.append((train_loader, val_loader))
                gene_dirs.append(gene_dir)
                best_val_losses.append(float("inf"))
                patience_counters.append(0)
                histories.append([])

            except Exception as e:
                print(f"Error loading gene {gene}: {str(e)}")

        for epoch in range(epochs):
            for i, (
                model,
                optimizer,
                criterion,
                (train_loader, val_loader),
            ) in enumerate(zip(models, optimizers, criterions, loaders)):
                gene = batch_genes[i]

                model.train()
                train_loss = 0.0
                for X_batch, y_batch in train_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    optimizer.zero_grad()
                    output = model(X_batch)
                    loss = criterion(output.squeeze(), y_batch)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()

                model.eval()
                val_loss = 0.0
                with torch.no_grad():
                    for X_batch, y_batch in val_loader:
                        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                        predictions = model(X_batch).squeeze()
                        val_loss += criterion(predictions, y_batch).item()

                if val_loss < best_val_losses[i]:
                    best_val_losses[i] = val_loss
                    patience_counters[i] = 0
                    torch.save(
                        model.state_dict(), os.path.join(gene_dirs[i], "best_model.pt")
                    )
                else:
                    patience_counters[i] += 1

                if patience_counters[i] >= patience:
                    break

            batch_pbar.update(1)

    batch_pbar.close()
    print("Training completed!")

In [40]:
data_dir = "processed_gene_data"
output_dir = "kan_models"

with open(os.path.join(data_dir, "gene_manifest.json"), "r") as f:
    genes_list = list(json.load(f).keys())

print(f"Found {len(genes_list)} genes to process")

train_kan_models_parallel(
    gene_list=genes_list,
    data_dir=data_dir,
    output_dir=output_dir,
    batch_size=32,
    max_models=4,
    epochs=50,
    patience=10,
    lr=0.001,
)

Found 23427 genes to process


Overall Progress:   0%|          | 0/23427 [00:00<?, ?it/s]


Preparing model for gene: BX293535.1
checkpoint directory created: ./model
saving model version 0.0

Preparing model for gene: WNT4
checkpoint directory created: ./model
saving model version 0.0

Preparing model for gene: ARL14
checkpoint directory created: ./model
saving model version 0.0

Preparing model for gene: IGLV1-47
checkpoint directory created: ./model
saving model version 0.0


KeyboardInterrupt: 